# Gerating artificial signal peptides

## 0 Introduction:
To avoid the combinatorial explosion that arises from the large number of possible amino acid sequences, an algorithm must be developed to narrow down the search space and identify the sequences that are most likely to function as signal peptides. This can be accomplished through a variety of computational methods, such as bioinformatics, machine learning, and statistical analysis.

One common approach is to use bioinformatics methods to analyze large sets of data on known signal peptides and identify patterns or features that are associated with signal peptide function. These features can then be used to predict the function of novel sequences.

Machine learning algorithms can also be used to predict signal peptides. These algorithms can be trained on large sets of data on known signal peptides, and can then be used to predict the function of novel sequences. Common machine learning algorithms used for this purpose include decision trees, random forests, and neural networks.

Another approach is to use statistical analysis to identify the regions of the peptide sequences that are most likely to function as signal peptides. This can be done by analyzing the frequency and distribution of different amino acids in known signal peptides and identifying those that are over-represented or under-represented in these sequences.

In summary, by developing an algorithm, we can narrow down the search space and identify the sequences that are most likely to function as signal peptides, thus avoiding combinatorial explosion. The algorithm we are showcasing here is based on a combination of bioinformatics, machine learning and statistical analysis.

### Agenda:
- Use AutoML predictions and synthetic signal peptide generation algorotihm to get a novel list of potential signal peptides

In [ ]:
import numpy as np
import pandas as pd
import random
import os

In [ ]:
def generate_artificial_peptide(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, max_length=68) -> str:
    """
    Generate an artificial peptide based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
        
    Returns:
    -------
    str
        Generated artificial peptide
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    out_str = ''
    for i in range(len(list_of_probabilities)):
        # make synthetic signal peptide
        artificial_amino_acid = list(np.random.choice(amino_acids, 1, p=list_of_probabilities[i]))

        if artificial_amino_acid == ['-']: 
            break

        out_str += artificial_amino_acid[0]
    return out_str


In [ ]:
def add_dunder_tail(peptide:str , max_lenght:int=22 ): 
    '''Adds a tail if a peptide is shorter than the specified max_len.
    '''
    if len(peptide) < max_lenght: 
        difference = max_lenght - len(peptide)
        sequence = peptide + ('-'*difference)
    else: 
        sequence = peptide
    
    
    return sequence        

In [ ]:
def generate_artificial_peptides(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, n_peptides: int, max_len=68) -> pd.DataFrame:
    """
    Generate a dataframe of artificial peptides based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
    n_peptides : int
        Number of peptides to generate
        
    Returns:
    -------
    pd.DataFrame
        Dataframe of generated artificial peptides with 'sequence' as column
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    artificial_peptides = []
    lengths = [] 
    for i in range(n_peptides): 
        peptide = generate_artificial_peptide(list_of_probabilities,amino_acids, max_length=max_len)
        if len(peptide) <= max_len:
            peptide_w_tail = add_dunder_tail(peptide, max_lenght = max_len)
        else: 
            continue
        
        # save
        lengths.append(len(peptide))                                     
        artificial_peptides.append(peptide_w_tail)

    df = pd.DataFrame(artificial_peptides, columns =['sequence'])
    df['length'] = lengths
    return df


In [ ]:
def split_peptides_sequences(df_signalPP:pd.DataFrame): 
    '''Split each AA for each position'''
    peptides_split = []
    for k,v in df_signalPP.iterrows(): 
        sequence = []
        for seq in v['sequence']: 
            sequence.append(seq)
        peptides_split.append(sequence)
    
    # make a dataframe
    new_peptides = pd.DataFrame(peptides_split)
    new_peptides = new_peptides.fillna('-')

    return new_peptides

## 1 AA probability matrix

Lets import our df_pwn dataframe that was made in a previous notebook and use the position specific scoring matrix for the generation of novel signal peptides

In [ ]:
from google.colab import drive 
drive.mount('/content/home')

Drive already mounted at /content/home; to attempt to forcibly remount, call drive.mount("/content/home", force_remount=True).


In [ ]:
df_pwn =  pd.read_csv('/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/Data/02_All_signal_peptides/df_pwn_68_positions.csv')
df_pwn

,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,Y,-
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.069877,0.001889,0.002833,0.001889,0.032106,0.021719,0.064212,0.037771,0.208687,0.084986,...,0.022663,0.028329,0.058546,0.193579,0.033994,0.018886,0.052880,0.014164,0.028329,0.000000
2,0.044381,0.007554,0.006610,0.007554,0.152975,0.047214,0.019830,0.050047,0.018886,0.190746,...,0.020774,0.052880,0.020774,0.053824,0.131256,0.053824,0.060434,0.025496,0.022663,0.000000
3,0.057602,0.001889,0.004721,0.004721,0.089707,0.019830,0.016053,0.064212,0.058546,0.175637,...,0.033994,0.041549,0.043437,0.056657,0.163362,0.100094,0.021719,0.010387,0.026440,0.000000
4,0.072710,0.004721,0.004721,0.007554,0.054769,0.032106,0.019830,0.077432,0.053824,0.155807,...,0.041549,0.040604,0.036827,0.045326,0.138810,0.096317,0.043437,0.023607,0.032106,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.001889,0.000944,0.000000,0.000000,0.000000,0.997167
59,0.000000,0.000000,0.000000,0.000944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000944,0.000000,0.000000,0.000000,0.998111
60,0.000944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000944,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.998111
61,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [ ]:
amino_acids = list(df_pwn.columns.values)

In [ ]:
list_of_probabilities = []
for i in range(len(df_pwn)): 
    list_of_probabilities.append(df_pwn.loc[i, :].values.tolist())

## 2 Load best model from AutoML

In [ ]:
%pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o.init(ip="localhost", min_mem_size_GB=8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.9/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmppt90n1qm
  JVM stdout: /tmp/tmppt90n1qm/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmppt90n1qm/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_unknownUser_sxvbeq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
h2o.__version__

'3.40.0.2'

In [ ]:
best_model = h2o.load_model("/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/Data/06_H2O_AutoML/06.1_Best_models/Second_round/m68_filtered_DeepLearning_grid_1_AutoML_1_20230312_144400_model_24.model")
best_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_1_20230312_144400_model_24


Status of Neuron Layers: predicting abundance, regression, gaussian distribution, Quadratic loss, 46,651 weights/biases, 559.1 KB, 1,973 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight             weight_rms           mean_bias             bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ---------------------  ----------  ----------------------  -------------------  --------------------  -----------------------
    1        931      Input             5.0
    2        50       RectifierDropout  40.0       0.0   0.0   0.14055796643409907    0.33713722229003906    0.0         -0.0008719045302109162  0.0467337965965271   0.4466184488938645    0.026746049523353577
    3        1        Linear                       0.0   0.0   0.0008273258749977685  0.0007948263082653284  0.0         -0.022937911627814174   0.17940837144851685  0.011832566911683641  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.0013900184429623498
RMSE: 0.03728295110318321
MAE: 0.006347276742007734
RMSLE: 0.028179884551363145
Mean Residual Deviance: 0.0013900184429623498

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.001395996280488333
RMSE: 0.037363033609281955
MAE: 0.004605881323683738
RMSLE: 0.02823483502928474
Mean Residual Deviance: 0.001395996280488333

Cross-Validation Metrics Summary: 
                        mean        sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  ----------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
mae                     0.00481809  0.00235062  0.0103722     0.00629833    0.00583005    0.00461052    0.00426714    0.00387951    0.00272853    0.00219862    0.0030325     0.00496343
mean_residual_deviance  0.00139153  0.00284296  0.00934221    0.00164352    0.00110241    0.000729027   0.00046062    0.000377913   0.000140797   1.10046e-05   5.63625e-05   5.14427e-05
mse                     0.00139153  0.00284296  0.00934221    0.00164352    0.00110241    0.000729027   0.00046062    0.000377913   0.000140797   1.10046e-05   5.63625e-05   5.14427e-05
r2                      -0.0792971  0.140746    -0.00949147   -0.00479519   -0.00349975   -0.0128475    -0.0163051    -0.0307302    -0.0121189    -0.213029     -0.0527087    -0.437446
residual_deviance       0.00139153  0.00284296  0.00934221    0.00164352    0.00110241    0.000729027   0.00046062    0.000377913   0.000140797   1.10046e-05   5.63625e-05   5.14427e-05
rmse                    0.0268163   0.0273336   0.0966551     0.0405403     0.0332025     0.0270005     0.0214621     0.01944       0.0118658     0.00331732    0.0075075     0.00717235
rmsle                   0.0219581   0.0186936   0.0670074     0.0338894     0.0286747     0.0238722     0.0194681     0.0178104     0.0112388     0.00329324    0.00728657    0.00704029

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples    training_rmse    training_deviance    training_mae    training_r2
--  -------------------  ----------  ----------------  --------  ------------  ---------  ---------------  -------------------  --------------  -------------
    2023-03-12 15:42:51  0.000 sec                     0         0             0          nan              nan                  nan             nan
    2023-03-12 15:42:51  33.895 sec  20340 obs/sec     1.85957   1             1973       0.037283         0.00139002           0.00634728      0.000489981

Variable Importance

## 3 Signal peptide predictor algorithm

In [ ]:
def signal_peptide_predictor(list_of_probabilities, amino_acids, n_peptides,  number_of_iterations:int)-> pd.DataFrame:
    '''Predicts best signal peptides from a number of iterations'''

    data = pd.DataFrame()
    for i in range(0,number_of_iterations):
        new_TO_NATURE_peptides = generate_artificial_peptides(list_of_probabilities, amino_acids, n_peptides=n_peptides, max_len = 68 )
        new_TO_NATURE_peptides = split_peptides_sequences(new_TO_NATURE_peptides)

        df_test = h2o.H2OFrame(pd.concat([new_TO_NATURE_peptides], axis='columns'))
        # make the df into categorical values
        for column in df_test.columns:
            if column != 'MM_N_peptide_abundance':
                df_test[column] = df_test[column].asfactor()

        #predict
        predicted = best_model.predict(df_test).as_data_frame()
        new_TO_NATURE_peptides['predictions'] = predicted['predict'].to_list()

        if len(data) == 0: 
            data = new_TO_NATURE_peptides.copy()
        else: 
            # concat to precious predictions
            data = pd.concat([data, new_TO_NATURE_peptides], axis=0)
            data = data.sort_values('predictions', ascending = False)
            data = data[0:1000]   

        if i % 1000 == 0:
            print(f"Iteration {i}")
            
    return data

In [ ]:
%%time
lets_predict_signal_peptides = signal_peptide_predictor(list_of_probabilities, 
                                                        amino_acids,n_peptides = 1000,  
                                                        number_of_iterations = 2000)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
lets_predict_signal_peptides

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,predictions
521,M,K,Q,I,S,F,R,G,L,Y,...,-,-,-,-,-,-,-,-,-,0.013179
541,M,K,L,I,Y,A,R,G,F,F,...,-,-,-,-,-,-,-,-,-,0.013154
350,M,A,L,V,P,P,P,N,L,L,...,-,-,-,-,-,-,-,-,-,0.013022
638,M,K,L,A,F,A,L,H,L,G,...,-,-,-,-,-,-,-,-,-,0.012890
177,M,Y,V,V,R,V,F,L,F,P,...,-,-,-,-,-,-,-,-,-,0.012869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,M,K,F,F,P,A,I,Q,L,F,...,-,-,-,-,-,-,-,-,-,0.007459
104,M,K,I,S,N,A,G,S,F,L,...,-,-,-,-,-,-,-,-,-,0.007458
289,M,A,L,F,R,R,A,L,A,F,...,-,-,-,-,-,-,-,-,-,0.007457
548,M,A,Q,T,S,W,L,L,A,Y,...,-,-,-,-,-,-,-,-,-,0.007455


In [ ]:
lets_predict_signal_peptides.to_excel('/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/Data/09_Generate_artificial_SPs/SPL68_144400_GASPs_0314.xlsx', index=False)
